In [1]:
import datajoint as dj

In [5]:
schema = dj.schema('common_test')

Connecting conrad@ucsd-demo-db.datajoint.io:3306


In [6]:
@schema
class Mouse(dj.Manual):
    definition = """
    #Mouse in the lab 
    mouse : char(8) #unique mouse id
    ---
    dob : date
    iacuc_barcode: bigint unsigned
    sex : enum('M','F')
    genotype : varchar(1000)
    
    unique index(iacuc_barcode)
    """

In [7]:
@schema
class MouseAlias(dj.Manual):
    definition="""
    ->Mouse
    ---
    alias : varchar(1000) #name of animal in code
    """

In [ ]:
MouseAlias.insert1(('CF_A1_05','a22'))

In [8]:
Mouse.insert1(dict(mouse='CF_D2_03',dob='2018-7-18',iacuc_barcode=00000,sex='M',genotype='C57/Bl6'))

DuplicateError: Duplicate entry 'CF_D2_03' for key 'PRIMARY' : To ignore duplicate entries, set skip_duplicates=True in insert.

In [ ]:
Mouse.insert1(dict(mouse='CF_A1_05',dob='2018-2-08',iacuc_barcode=2,sex='M',genotype='C57/Bl6'))

In [ ]:
@schema
class Session(dj.Manual):
    definition="""
    ->Mouse
    session : tinyint unsigned
    ---
    path_to_mat : varchar(1000) #Two photon tiff file
    type : enum('baseline','fbd1','fbd2','post')
    """

In [ ]:
types = ['baseline','fbd1','fbd2','post']
for i in range(4):
    Session.insert1(('CF_D2_03',i,"/media/conrad/USB DISK/Feedback animals for papers/Analysis objects/d2_animal_structs_5.mat",types[i]))

In [10]:
@schema
class Behavior(dj.Imported):
    definition="""
    ->Session
    ---
    licking : longblob
    velocity : longblob
    time : longblob
    """
    
    def make(self,key):
        path = (Session & key).fetch1('path_to_mat')
        type_ = (Session & key).fetch1('type')
        alias = (MouseAlias & key).fetch1('alias')
        with h5py.File(path) as f:
            self.insert1(dict(key,licking=np.array(f.get(alias).get(type_).get('lab').get('lick')),velocity=np.array(f.get(alias).get(type_).get('lab').get('vel')),time=np.array(f.get(alias).get(type_).get('lab').get('t'))))
        

In [ ]:
Behavior.populate(order="reverse")

In [11]:
Behavior()

mouse unique mouse id,session,licking,velocity,time
CF_D2_03,0,=BLOB=,=BLOB=,=BLOB=
CF_D2_03,1,=BLOB=,=BLOB=,=BLOB=
CF_D2_03,2,=BLOB=,=BLOB=,=BLOB=
CF_D2_03,3,=BLOB=,=BLOB=,=BLOB=
